In [ ]:
import torch

from semantic_memory import vsm
from tqdm import trange

In [ ]:
embeddings = []
with open("spose_embedding_66d_sorted.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        vector = [float(x) for x in line.split()]
        embeddings.append(vector)

embeddings = torch.tensor(embeddings)

In [ ]:
vocab = open("unique_id.txt", "r").readlines()
vocab = [x.strip() for x in vocab]

In [ ]:
things = vsm.VectorSpaceModel("THINGS 66d")
things.load_vectors_from_tensor(embeddings, vocab)

In [ ]:
things.neighbor("mouse1",k=10)

In [ ]:
def dot_product_similarity(v1, v2):
    return torch.dot(v1, v2)

In [ ]:
dot_product_similarity(things("mouse1").squeeze(0), things("rat").squeeze(0))

In [ ]:
def embedding2sim(embedding):
    # Convert embedding to similarity matrix
    n_objects = embedding.size(0)

    # Compute the similarity matrix
    sim = torch.mm(embedding, embedding.T)
    esim = torch.exp(sim)

    print("Initialized similarities")

    # Initialize the cp matrix
    cp = torch.zeros(n_objects, n_objects)

    print("Initialized matrix")

    # Compute the similarity matrix
    for i in range(n_objects):
        for j in trange(i + 1, n_objects):
            ctmp = torch.zeros(n_objects)
            for k in range(n_objects):
                if k == i or k == j:
                    continue
                ctmp[k] = esim[i, j] / (esim[i, j] + esim[i, k] + esim[j, k])
            cp[i, j] = torch.sum(ctmp)

    # Normalize the cp matrix
    cp = cp / (n_objects - 2)
    cp = cp + cp.T  # Make the matrix symmetric
    cp[torch.eye(n_objects).bool()] = 1  # Set the diagonal to 1

    return cp

In [ ]:
spose_sim = embedding2sim(embeddings)

In [ ]:
n_objects = embeddings.size(0)
c=0
for i in range(n_objects):
    for j in range(i + 1, n_objects):
        c+=1
        # ctmp = torch.zeros(n_objects)
        # for k in range(n_objects):

In [ ]:
c